<a href="https://colab.research.google.com/github/andBabaev/awesome-action-recognition/blob/master/flask_colab_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok

In [2]:
!pip install pafy
!pip install youtube_dl --upgrade

     |████████████████████████████████| 1.8MB 3.5MB/s 


In [3]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, Response, request, url_for

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, TimeDistributed, LSTM, Input, Dense, Activation, Dropout
from tensorflow.keras.layers import  Multiply, Lambda, Flatten, Concatenate, Permute, Conv2D,Reshape, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

import numpy as np
import cv2
import pafy
import time
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns
import os
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
import tensorflow_hub as hub
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
target_classes = ['bicycling', 'cooking', 'dancing', 'drawing', 'drinking',
                  'eating', 'crying', 'walking', 'exercising', 'jumping']
n_channels = 3
len_seq = 10 
fps = 5
image_shape = (224,224)

#model = load_model('/content/drive/My Drive/roonyx_models/Mobnet_overfitting')

inputs = Input((len_seq, *image_shape,n_channels))
base_model = MobileNetV2(weights='imagenet',include_top=False, input_shape=(*image_shape,n_channels))
outputs = TimeDistributed(base_model)(inputs)
x = TimeDistributed(GlobalAveragePooling2D())(outputs)
x = LSTM(128, dropout=0.5, return_sequences=True)(x)
x = LSTM(128)(x)
preds = Dense(len(target_classes), activation='softmax')(x)
model = Model(inputs,preds)

model.load_weights('/content/drive/My Drive/roonyx_models/mobnet_2lstm128_mit_full_final_weights/full_model_weights-06-0.61.h5')
print('Model loading is completed')

model.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

9412608/9406464 [==============================] - 0s 0us/step
Model loading is completed


In [0]:
heatmap_model = tf.keras.models.Model(
    [model.inputs], [model.layers[1].output, model.output]
)

waling bad https://www.youtube.com/watch?v=T4h1t7QQY64

walking good https://www.youtube.com/watch?v=NyLF8nHIquM

bicycling  https://www.youtube.com/watch?v=UdTu8v6cWVo

cooking https://www.youtube.com/watch?v=6DOgPETTyoU

drawing https://www.youtube.com/watch?v=GMQPiO7Xfw8

drinking https://www.youtube.com/watch?v=Lw1xWh-z-nA

crying https://www.youtube.com/watch?v=ee925OTFBCA

training https://www.youtube.com/watch?v=5nZ8n1vUqVU https://www.youtube.com/watch?v=_K7rv_vFOWM

dancing https://www.youtube.com/watch?v=uUGsLoZYXb4

eating https://www.youtube.com/watch?v=eO8oUffeeIY

In [7]:
!mkdir 'templates/'
!mkdir 'static/'
!mkdir 'static/css/'
# t = '''
# <html>
#   <head>
#     <title>Demo</title>
#   </head>
#   <body>
#     <h1 align=center>Action Recognition Demonstration</h1>
#     <div align=center>
#         <img id="bg" src="{{ video_output }}">
#     </div>
#     <form align=center method="POST">
#         <input name="text" value="https://www.youtube.com/watch?v=UdTu8v6cWVo">
#         <input type="submit">
#     </form>
#   </body>
# </html>'''
# with open('templates/index.html', 'w') as f:
#     f.write(t)

mkdir: cannot create directory ‘templates/’: File exists


In [89]:
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
CAM_visible = False
all_scores_visible = True

def gen(url):
    X = np.empty((0, *image_shape, n_channels))
    current_action = 'No action'
    #scale_coef = 0.5
    alpha=0.5
    colormap=cv2.COLORMAP_VIRIDIS

    vPafy = pafy.new(url)
    play = vPafy.getbest(preftype="mp4")
    video = cv2.VideoCapture(play.url)
    
    frame_step = int(video.get(cv2.CAP_PROP_FPS) / fps)+1
    wait_time = 1/video.get(cv2.CAP_PROP_FPS)
    temp_step = 0
    print(wait_time)
    success, frame = video.read()
    heatmap = np.zeros_like(frame)
    
    
    if frame.shape[0] < 400:
        scale_coef = 400 / frame.shape[0] 
    elif frame.shape[0] > 720:
        scale_coef = 720 / frame.shape[0]
    else:
        scale_coef = 1.0
    print(frame.shape)

    #black_area = np.full((frame.shape[0],300, 3), 0)
    preds = None
    while True:

        success, frame = video.read()
        if success:
            if temp_step % frame_step == 0:
                X = np.vstack((X, cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 
                                        image_shape, 
                                        interpolation=cv2.INTER_AREA)[np.newaxis,...]))
                temp_step = 0
            temp_step += 1

            if X.shape[0] == len_seq:

                if not CAM_visible:
                    preds = model.predict(X[np.newaxis,...]/255.0)
                    heatmap = np.zeros_like(frame)
                else:

                    with tf.GradientTape() as tape:
                        conv_output, preds = heatmap_model(X[np.newaxis,...]/255.)
                        loss = preds[:, np.argmax(preds[0])]
                    grads = tape.gradient(loss, conv_output)[:, -1]
                    conv_output = conv_output[:, -1]
                    
                    castConvOutputs = tf.cast(conv_output > 0, "float32")
                    castGrads = tf.cast(grads > 0, "float32")
                    guidedGrads = castConvOutputs * castGrads * grads
                    
                    conv_output = conv_output[0]
                    guidedGrads = guidedGrads[0]

                    weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
                    cam = tf.reduce_sum(tf.multiply(weights, conv_output), axis=-1)

                    (w, h) = (frame.shape[1], frame.shape[0])
                    heatmap = cv2.resize(cam.numpy(), (w, h))

                    eps=1e-8
                    numer = heatmap - np.min(heatmap)
                    denom = (heatmap.max() - heatmap.min()) + eps
                    heatmap = numer / denom
                    heatmap = (heatmap * 255).astype("uint8")
                    heatmap = cv2.applyColorMap(heatmap, colormap)
                    

                preds = preds[0]
                if np.max(preds) >= 0.55:
                    current_action = target_classes[np.argmax(preds)] + ' ' + str(np.round(np.max(preds), 2))
                else:
                    current_action = 'No action'
                X = X[1:]
            else:
                time.sleep(wait_time)
            
            frame = cv2.addWeighted(frame.astype("uint8"), alpha, heatmap, 1 - alpha, 0,)
            #frame = np.hstack([black_area, frame])
            frame = cv2.rectangle(frame, (0,0), (300,50), (0,0,0), -1)

                        
            
            if frame.shape[0] > 1.0:
                frame = cv2.resize(frame, None, fx=scale_coef, fy=scale_coef, 
                                   interpolation=cv2.INTER_AREA)

            frame = cv2.putText(frame, current_action, (25, 30), cv2.FONT_HERSHEY_SIMPLEX ,  
                            1, (255, 255, 255) , 2, cv2.LINE_AA) 
            if preds is not None and all_scores_visible:
                for i, (class_, score) in enumerate(zip(target_classes, preds), 1):
                    text = class_ + ' ' + str(np.round(score, 2))
                    frame = cv2.putText(frame, text, (25, 60+27*i), cv2.FONT_HERSHEY_SIMPLEX ,  
                                0.8, (255, 255, 255) , 2, cv2.LINE_AA) 

            #frame = cv2.resize(frame, None, fx=scale_coef, fy=scale_coef, interpolation=cv2.INTER_AREA)
            if frame.shape[0] < 1.0:
                frame = cv2.resize(frame, None, fx=scale_coef, fy=scale_coef, 
                                   interpolation=cv2.INTER_AREA)

            ret, jpeg = cv2.imencode('.jpg', frame)
            frame = jpeg.tobytes()
        else:
            video.release()
            break

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')






@app.route('/')
def index():
    # rendering webpage
    return render_template('index.html', 
                           video_output='static/start_rec.jpg', 
                           box_cam=CAM_visible, 
                           box_scores=all_scores_visible,
                           url_video='https://www.youtube.com/watch?v=UdTu8v6cWVo')

@app.route('/video_feed/<path:url>')
def video_feed(url):
    #url = 'https://www.youtube.com/watch?v=UdTu8v6cWVo'
    return Response(gen(url),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
  
@app.route('/form_url', methods=['POST'])
def form_url():
    url_ = request.form['text']
    if 'youtube' in url_:
        new_src = url_for('video_feed', url=url_)
        return render_template('index.html',
                            video_output=new_src, 
                            box_cam=CAM_visible, 
                            box_scores=all_scores_visible,
                            url_video=url_)
    else:
        return '', 204
    
@app.route('/form_cam', methods=['POST'])
def form_cam():
    global CAM_visible, all_scores_visible
    CAM_visible = True if request.form.get('activator') is not None else False
    all_scores_visible = True if request.form.get('all_scores') is not None else False
    return '', 204

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://157fdb296e59.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Jun/2020 15:55:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 15:55:58] "GET /static/css/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 15:55:58] "GET /static/start_rec.jpg HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 15:55:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Jun/2020 15:56:01] "POST /form_url HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 15:56:02] "GET /video_feed/https://www.youtube.com/watch%3Fv%3DUdTu8v6cWVo HTTP/1.1" 200 -


0.04170833333333333
(720, 1280, 3)


127.0.0.1 - - [10/Jun/2020 15:56:14] "POST /form_url HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 15:56:16] "GET /video_feed/https://www.youtube.com/watch%3Fv%3DLw1xWh-z-nA HTTP/1.1" 200 -


0.03333333333333333
(720, 406, 3)


127.0.0.1 - - [10/Jun/2020 15:56:27] "POST /form_cam HTTP/1.1" 204 -
127.0.0.1 - - [10/Jun/2020 15:56:27] "POST /form_cam HTTP/1.1" 204 -


# HMDB51

In [0]:
target_classes = ['fall_floor', 'punch', 'smoke', 'sit', 'walk']
n_channels = 3
len_seq = 10 
fps = 5
image_shape = (224,224)
seqShape = (len_seq, *image_shape,n_channels)
inputs = Input(seqShape)
base_model = MobileNetV2(weights='imagenet',include_top=False, input_shape=(*image_shape,n_channels))
outputs = TimeDistributed(base_model)(inputs)
x = Permute((2, 3, 4, 1))(outputs)
x = Reshape((7,7,1280*len_seq))(x)
x = Conv2D(1280, (1,1), padding='same', activation='relu')(x)

inputs_1 = Input((*image_shape,n_channels))
base_model_1 = MobileNetV2(weights='imagenet',include_top=False, input_shape=(*image_shape,n_channels))
outputs_1 = base_model_1(inputs_1)

x = Concatenate(axis=3)([x, outputs_1])
x = Conv2D(1024, (1,1), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(1024, (3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)

def f(x):
    F_T = Reshape((49, 1024))(x)
    F = tf.transpose(F_T, perm=(0,2,1))
    F_T = tf.linalg.matmul(F, F_T)
    M = tf.nn.softmax(F_T)
    F_ = tf.linalg.matmul(M, F)
    F_ = tf.transpose(F_, perm=(0,2,1))
    F__ = Reshape((7,7,1024))(F_)
    return x+F__

x = Lambda(f)(x)
x = Conv2D(1024, (3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(1024, (1,1), padding='same', activation='relu')(x)

x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
preds = Dense(len(target_classes), activation='softmax')(x)

model = Model([inputs, inputs_1], preds)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 10, 224, 224 0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 10, 7, 7, 128 2257984     input_19[0][0]                   
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 7, 7, 1280, 1 0           time_distributed_1[0][0]         
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 7, 7, 12800)  0           permute_1[0][0]                  
____________________________________________________________________________________________

In [0]:
for layer in base_model.layers[:135]:
    layer.trainable = False
for layer in base_model.layers[135:]:
    layer.trainable = True
for layer in base_model_1.layers[:135]:
    layer.trainable = False
for layer in base_model_1.layers[135:]:
    layer.trainable = True
model.load_weights('/content/drive/My Drive/roonyx_models/Mobnet_yowo_hmdb/Mobnet_yowo__hmdb51_final-43-0.64.h5')

In [0]:
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
def gen(url):
    X = np.empty((0, *image_shape, n_channels))
    current_action = 'No action'
    scale_coef = 0.5
    alpha=0.5
    colormap=cv2.COLORMAP_VIRIDIS

    vPafy = pafy.new(url)
    play = vPafy.getbest(preftype="mp4")
    video = cv2.VideoCapture(play.url)
    
    frame_step = int(video.get(cv2.CAP_PROP_FPS) / fps)+1
    wait_time = 1/video.get(cv2.CAP_PROP_FPS)
    temp_step = 0

    success, frame = video.read()
    heatmap = np.zeros_like(frame)
    while True:

        success, frame = video.read()
        if success:
            if temp_step % frame_step == 0:
                X = np.vstack((X, cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 
                                        image_shape, 
                                        interpolation=cv2.INTER_AREA)[np.newaxis,...]))
                temp_step = 0
            temp_step += 1

            if X.shape[0] == len_seq:
                X_temp = preprocess_input(X)[np.newaxis,...]
                preds = model.predict([X_temp, X_temp[:,-1]])
                               

                preds = preds[0]
                if np.max(preds) >= 0.55:
                    current_action = target_classes[np.argmax(preds)] + str(np.round(np.max(preds), 2))
                else:
                    current_action = 'No action'
                X = X[1:]
            else:
                time.sleep(wait_time)
            
            frame = cv2.addWeighted(frame.astype("uint8"), alpha, heatmap, 1 - alpha, 0,)
            frame = cv2.rectangle(frame, (0,0), (300,50), (0,0,0), -1)
            frame = cv2.putText(frame, current_action, (25, 30), cv2.FONT_HERSHEY_SIMPLEX ,  
                            1, (255, 255, 255) , 2, cv2.LINE_AA) 
            frame = cv2.resize(frame, None, fx=scale_coef, fy=scale_coef, interpolation=cv2.INTER_AREA)
            
            ret, jpeg = cv2.imencode('.jpg', frame)
            frame = jpeg.tobytes()
        else:
            video.release()
            break

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

@app.route('/')
def index():
    # rendering webpage
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    url = 'https://www.youtube.com/watch?v=UdTu8v6cWVo'
    return Response(gen(url),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://87dc236832a5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Jun/2020 16:02:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 16:02:57] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2020 16:02:59] "GET /favicon.ico HTTP/1.1" 404 -
